In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pickle
from pyprojroot import here
from mpl_lego.style import use_latex_style
from mpl_lego.labels import bold_text, apply_subplot_labels

from llm_deliberation.multilevel import (verdict2num,
                                         build_all_data,
                                         fit_map_blended,
                                         fit_map_split,
                                         bootstrap_split)

In [ ]:
use_latex_style()

In [ ]:
with open(here('data/analysis/exp1_round_robin_3way.pkl'), 'rb') as file:
    gem_cla_gpt = pickle.load(file)

with open(here('data/analysis/exp2_round_robin_3way.pkl'), 'rb') as file:
    cla_gem_gpt = pickle.load(file)

with open(here('data/analysis/exp3_round_robin_3way.pkl'), 'rb') as file:
    gpt_cla_gem = pickle.load(file)

with open(here('data/analysis/exp4_round_robin_3way.pkl'), 'rb') as file:
    gpt_gem_cla = pickle.load(file)

with open(here('data/analysis/exp5_round_robin_3way.pkl'), 'rb') as file:
    gem_gpt_cla = pickle.load(file)

with open(here('data/analysis/exp6_round_robin_3way.pkl'), 'rb') as file:
    cla_gpt_gem = pickle.load(file)

In [ ]:
def rounds_to_display(df: pd.DataFrame, normalize: bool = True) -> pd.Series:
    """
    Combine `n_rounds` and `final_verdict` to produce the display series:
    1/2/3 map through unchanged, 4 resolves to 4 or 'No Consensus'.
    """
    outcome = df['n_rounds'].astype('object').copy()
    mask_four = outcome == 4
    # treat both None and NaN as “no consensus”
    no_consensus = df['final_verdict'].isna() | (df['final_verdict'].isna())
    outcome[mask_four & no_consensus] = 'No Consensus'
    counts = outcome.value_counts(normalize=normalize)
    if 'No Consensus' not in counts:
        counts['No Consensus'] = 0
    return counts.loc[[1, 2, 3, 4, 'No Consensus']]

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(8, 4), sharex=True, sharey=True)
plt.subplots_adjust(hspace=0.6)
axes_list = axes.ravel()

LABELS = {'gpt': 'GPT', 'cla': 'Claude', 'gem': 'Gemini'}
exp_and_titles = [
        (gpt_cla_gem, '(1) GPT\n(2) Claude\n(3) Gemini'),
        (cla_gpt_gem, '(1) Claude\n(2) GPT\n(3) Gemini'),
        (gem_gpt_cla, '(1) Gemini\n(2) GPT\n(3) Claude'),
        (gpt_gem_cla, '(1) GPT\n(2) Gemini\n(3) Claude'),
        (cla_gem_gpt, '(1) Claude\n(2) Gemini\n(3) GPT'),
        (gem_cla_gpt, '(1) Gemini\n(2) Claude\n(3) GPT'),
    ]

for ax, (exp, title) in zip(axes_list, exp_and_titles):
    rounds_to_display(exp).plot(ax=ax, kind='bar', color='slategray', edgecolor='black')
    ax.tick_params(axis='x', rotation=0)
    ax.set_xticklabels(['1', '2', '3', '4', 'No\nConsensus'])
    ax.set_ylim([0, 1])
    ax.grid('on', axis='y')
    ax.set_axisbelow(True)
    ax.set_title(bold_text(title), fontsize=10)

for ax in axes[1]:
    ax.set_xlabel(bold_text('Number of Rounds'))
for ax in axes[:, 0]:
    ax.set_ylabel(bold_text('Fraction of Dilemmas'))

apply_subplot_labels(axes, x=-0.05, y=1.13, size=13, bold=True)
plt.savefig('figure_n_rounds_3way.pdf', bbox_inches='tight')